In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import datetime
import pprint
import tqdm
# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

c:\Users\Benny\miniconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class FFN(tf.keras.Model):
    def __init__(self):
        super().__init__()
    
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
        
        self.metrics_list = [
                        tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
                        tf.keras.metrics.TopKCategoricalAccuracy(3,name="top-3-acc") 
                       ]
        
        self.loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False)   
        
        # define layers
        self.layer1 = tf.keras.layers.Dense(256,activation=tf.nn.relu)
        self.layer2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    
    def call(self, x, training=False):
        x = self.layer1(x)
        x = self.layer2(x)
        out = self.output_layer(x)
       
        return out
    
    def reset_metrics(self):
        
        for metric in self.metrics:
            metric.reset_states()
            
    @tf.function
    def train_step(self, data):
        
        x, targets = data
        
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            
            loss = self.loss_function(targets, predictions) + tf.reduce_sum(self.losses)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(targets,predictions)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):

        x, targets = data
        predictions = self(x, training=False)
        loss = self.loss_function(targets, predictions) + tf.reduce_sum(self.losses)

        self.metrics[0].update_state(loss)
        # for accuracy metrics:
        for metric in self.metrics[1:]:
            metric.update_state(targets, predictions)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
train_ds, val_ds = tfds.load("mnist", split=["train", "test"], as_supervised=True)

train_ds = train_ds.map(lambda img, target:)

In [4]:
train_ds, val_ds = tfds.load("mnist", split=["train", "test"], as_supervised=True)

train_ds = train_ds.map(lambda img, target: ((tf.cast(tf.reshape(img, (-1,)), tf.float32)/128.)-1., tf.one_hot(target, 10, dtype=tf.float32)),\
                        num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(5000).batch(32).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda img, target: ((tf.cast(tf.reshape(img, (-1,)), tf.float32)/128.)-1., tf.one_hot(target, 10, dtype=tf.float32)),\
                    num_parallel_calls=tf.data.AUTOTUNE).cache().batch(32).prefetch(tf.data.AUTOTUNE)


In [4]:
list(train_ds.take(1))

[(<tf.Tensor: shape=(32, 784), dtype=float32, numpy=
  array([[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32)>,
  <tf.Tensor: shape=(32, 10), dtype=float32, numpy=
  array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 

In [5]:
# instantiate the model
model = FFN()

# run model on input once so the layers are built
model(tf.keras.Input((32, 784)))
model.summary()

Model: "ffn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  200960    
                                                                 
 dense_1 (Dense)             multiple                  65792     
                                                                 
 dense_2 (Dense)             multiple                  2570      
                                                                 
Total params: 269,328
Trainable params: 269,322
Non-trainable params: 6
_________________________________________________________________


In [6]:
# Define where to save the log
config_name= "config_name"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
val_log_path = f"logs/{config_name}/{current_time}/val"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
val_summary_writer = tf.summary.create_file_writer(val_log_path)

In [7]:
def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer):
    for epoch in range(epochs):
        print(f"Epoch {epoch}:")
        
        # Training:
        tq = tqdm.tqdm(train_ds, position=0, leave=True)
        for data in tq:
            metrics = model.train_step(data)
            # logging the validation metrics to the log file which is used by tensorboard
            with train_summary_writer.as_default():
                for metric in model.metrics:

                    # if metric
                    # tq.set_description(f"MSE: {}",refresh=True)
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics (requires a reset_metrics method in the model)
        model.reset_metrics()    
        
        # Validation:
        for data in val_ds:
            metrics = model.test_step(data)
        
            # logging the validation metrics to the log file which is used by tensorboard
            with val_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
                    
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        model.reset_metrics()
        print("\n")

In [8]:
%tensorboard --logdir logs/

Reusing TensorBoard on port 6006 (pid 16280), started 1:27:34 ago. (Use '!kill 16280' to kill it.)

In [9]:
# run the training loop 
training_loop(model=model, 
                train_ds=train_ds, 
                val_ds=val_ds, 
                epochs=50, 
                train_summary_writer=train_summary_writer, 
                val_summary_writer=val_summary_writer)

Epoch 0:


100%|██████████| 1875/1875 [00:26<00:00, 71.94it/s]


['loss: 0.34150853753089905', 'acc: 0.8912166953086853', 'top-3-acc: 0.9734166860580444']
['val_loss: 0.15371127426624298', 'val_acc: 0.9517999887466431', 'val_top-3-acc: 0.9930999875068665']


Epoch 1:


100%|██████████| 1875/1875 [00:26<00:00, 70.42it/s]


['loss: 0.13463154435157776', 'acc: 0.9584833383560181', 'top-3-acc: 0.994949996471405']
['val_loss: 0.1248740628361702', 'val_acc: 0.9607999920845032', 'val_top-3-acc: 0.9939000010490417']


Epoch 2:


...

['loss: 4.9554233555682003e-05', 'acc: 1.0', 'top-3-acc: 1.0']
['val_loss: 0.08892601728439331', 'val_acc: 0.984000027179718', 'val_top-3-acc: 0.9986000061035156']


Epoch 48:


100%|██████████| 1875/1875 [00:23<00:00, 79.66it/s]


['loss: 4.811776307178661e-05', 'acc: 1.0', 'top-3-acc: 1.0']
['val_loss: 0.08911961317062378', 'val_acc: 0.9839000105857849', 'val_top-3-acc: 0.9986000061035156']


Epoch 49:


100%|██████████| 1875/1875 [00:23<00:00, 80.34it/s]


['loss: 4.651385097531602e-05', 'acc: 1.0', 'top-3-acc: 1.0']
['val_loss: 0.08929443359375', 'val_acc: 0.9839000105857849', 'val_top-3-acc: 0.9986000061035156']




In [10]:
# save the model with a meaningful name
model.save_weights(f"saved_model_{config_name}", save_format="tf")

# load the model:
# instantiate a new model from our CNN class
loaded_model = FFN()

# build the model
inp= tf.keras.Input((28,28,1))
loaded_model(inp)

# load the model weights to continue training. 
loaded_model.load_weights(f"saved_model_{config_name}");

# continue training (but: optimizer state is lost)

# run the training loop 
training_loop(model=loaded_model, 
                train_ds=train_ds, 
                val_ds=val_ds, 
                epochs=10, 
                train_summary_writer=train_summary_writer, 
                val_summary_writer=val_summary_writer)

ValueError: Received incompatible tensor with shape (784, 256) when attempting to restore variable with shape (1, 256) and name layer1/kernel/.ATTRIBUTES/VARIABLE_VALUE.